## Importación de las librerías necesarias

In [31]:
import numpy as np
import pandas as pd
from datetime import datetime

## ETL

### Dataset de la USGS - Período 01/01/2013-Now

OBSERVACIONES:  
* La USGS (United States Geological Survey) recopila y proporciona datos sobre sismos de diversos países, no solo de Estados Unidos, entre los cuales se encuentra Japón y Perú.
* Se ha establecido un intervalo de petición de 10 días debido a que la API de la página solo permite obtener un máximo de 20,000 registros por solicitud. En algunos casos, grupos de datos de hasta 15 días contienen más de 20,000 registros, lo que provoca que la API muestre el error 400.

In [ ]:
# Extracción de datos de la API del USGS
start_date = pd.to_datetime("2013-01-01")
end_date = pd.to_datetime(datetime.now())
interval = pd.DateOffset(days=10)
dfs = []
while start_date <= end_date:
    next_date = start_date + interval
    url = f"https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&starttime={start_date.strftime('%Y-%m-%d')}&endtime={next_date.strftime('%Y-%m-%d')}"
    df = pd.read_csv(url).sort_values(by="time")
    dfs.append(df)
    start_date+=interval

df_usgs = pd.concat(dfs)

# P.D: El tiempo de extracción de los 10 últimos años es de aprox. 1 hora

In [9]:
df_usgs.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2013-01-01T00:08:44.056Z,64.9725,-147.3531,0.0,1.3,ml,NaN,NaN,NaN,0.2600,...,2019-05-13T22:42:25.785Z,"12 km E of Fox, Alaska",explosion,NaN,0.0,NaN,NaN,reviewed,ak,ak
1,2013-01-01T00:10:51.379Z,40.1074,-119.6768,14.6,-0.9,ml,3.0,137.68,0.076,0.0053,...,2018-06-29T22:31:38.899Z,"18 km NNW of Sutcliffe, Nevada",earthquake,NaN,5.5,0.29,3.0,reviewed,nn,nn
2,2013-01-01T00:14:16.890Z,51.9750,-169.5660,87.7,2.9,ml,7.0,228.10,NaN,NaN,...,2019-05-13T22:42:39.803Z,"117 km SSW of Nikolski, Alaska",earthquake,NaN,NaN,NaN,NaN,reviewed,aeic,aeic
3,2013-01-01T00:21:07.729Z,61.6839,-147.4021,29.7,1.6,ml,NaN,NaN,NaN,0.8100,...,2019-05-13T22:42:26.097Z,"19 km SE of Glacier View, Alaska",earthquake,NaN,0.2,NaN,NaN,reviewed,ak,ak
4,2013-01-01T00:24:33.234Z,61.3167,-151.3511,73.3,1.3,ml,NaN,NaN,NaN,0.2300,...,2019-05-13T22:42:40.207Z,"24 km NW of Beluga, Alaska",earthquake,NaN,0.7,NaN,NaN,reviewed,ak,ak


In [10]:
# Información del Dataset, incluyendo la cantidad de nulos por columna y el número total de registros y columnas.
df_usgs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1609453 entries, 0 to 1609452
Data columns (total 22 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   time             1609453 non-null  object 
 1   latitude         1609453 non-null  float64
 2   longitude        1609453 non-null  float64
 3   depth            1609451 non-null  float64
 4   mag              1606407 non-null  float64
 5   magType          1606401 non-null  object 
 6   nst              1013682 non-null  float64
 7   gap              1183379 non-null  float64
 8   dmin             1081212 non-null  float64
 9   rms              1606691 non-null  float64
 10  net              1609453 non-null  object 
 11  id               1609453 non-null  object 
 12  updated          1609453 non-null  object 
 13  place            1604264 non-null  object 
 14  type             1609453 non-null  object 
 15  horizontalError  1008492 non-null  float64
 16  depthError       1

In [ ]:
# No existen registros duplicados
df_usgs[df_usgs.duplicated(keep=False)]

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,country_estate


#### Diccionario de Datos:
- 'time': Indica la fecha en que ocurrió el sismo.

- 'latitude': Representa la latitud geográfica del epicentro del sismo.

- 'longitude': Representa la longitud geográfica del epicentro del sismo.

- 'depth': Indica la profundidad en kilómetros desde la superficie terrestre a la que se produjo el sismo.

- 'mag': Es la magnitud del sismo, que es una medida cuantitativa de la energía liberada durante el evento sísmico.

- 'magType': Especifica el tipo de escala utilizada para medir la magnitud del sismo.

- 'nst': Representa el número de estaciones sismográficas utilizadas para calcular la magnitud del sismo.

- 'gap': Es el ángulo máximo entre las estaciones sismográficas para estimar la magnitud del sismo.

- 'dmin': Es la distancia mínima, en grados, entre el epicentro del sismo y la estación sismográfica más cercana.

- 'rms': Es el error estimado del ajuste en la magnitud del sismo.

- 'net': Indica la red sismográfica o la agencia responsable de monitorear y reportar el sismo.

- 'id': Es un identificador único asignado a cada evento sísmico.

- 'updated': Indica la fecha y hora de la última actualización de la información del sismo en el dataset.

- 'place': Proporciona la ubicación geográfica general donde ocurrió el sismo, como el nombre del país, estado o ciudad.

- 'type': Especifica el tipo de evento sísmico.

- 'horizontalError': Indica el error estimado en la ubicación horizontal del epicentro del sismo.

- 'depthError': Representa el error estimado en la medida de la profundidad del sismo.

- 'magError': Es el error estimado en la magnitud del sismo.

- 'magNst': Indica el número de estaciones sismográficas utilizadas para calcular la magnitud del sismo.

- 'status': Indica el estado o la condición del evento sísmico.

- 'locationSource': Es la fuente o la agencia responsable de proporcionar la información sobre la ubicación del sismo.

- 'magSource': Indica la fuente o la agencia responsable de proporcionar la información sobre la magnitud del sismo.

#### Eliminación de columnas innecesarias

In [11]:
# Estas variables son eliminadas debido a que no se utilizarán en ninguna etapa del proyecto

df_usgs.drop(columns=["nst","gap","dmin","rms","net","id","depthError","magError","magNst",
                      "status","locationSource","magSource"],inplace=True)

#### Conversión de las variables de fecha a tipo datetime

In [13]:
df_usgs["time"]=pd.to_datetime(df_usgs.time)
df_usgs["updated"]=pd.to_datetime(df_usgs.updated)

#### Eliminación de los valores faltantes de la columna `depth`

In [16]:
# Cantidad de nulos
df_usgs[df_usgs.depth.isnull()].shape[0]

2

In [17]:
df_usgs.dropna(subset="depth",inplace=True)

#### Imputación de valores faltantes de la variable `place`

In [37]:
df_usgs["place"]=df_usgs.place.fillna("Unknown")

#### Eliminación de los registros que no son terremotos en `type`

In [47]:
# Porcentaje de registros que son terremotos
round((df_usgs[df_usgs.type=="earthquake"].shape[0]/df_usgs.shape[0])*100,2)

97.63

In [ ]:
df_usgs=df_usgs[df_usgs.type=="earthquake"]

# Eliminación de la columna 'type'
df_usgs.drop(columns="type",inplace=True)

#### Imputación de valores faltantes de la variable `horizontalError`

In [42]:
df_usgs["horizontalError"]=df_usgs.horizontalError.fillna(0)

#### Creación de una columna que contenga el nombre del país o estado

In [45]:
df_usgs["country_estate"]=df_usgs.place.apply(lambda x: x.split(", ")[-1])

#### Creación de los datasets de Estados Unidos, Japón y Perú

In [52]:
### Estados Unidos
# Lista de estados de E.E.U.U y sus abreviaturas
state = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'North Carolina', 'South Carolina', 'Colorado',
           'Connecticut', 'North Dakota', 'South Dakota', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois',
           'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan',
           'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey',
           'New York', 'New Mexico', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'Tennessee', 'Texas',
           'Utah', 'Vermont', 'Virginia', 'West Virginia', 'Washington', 'Wisconsin', 'Wyoming', 'District of Columbia']

abbreviation = ['AL', 'AK', 'AZ', 'AR', 'CA', 'NC', 'SC', 'CO', 'CT', 'ND', 'SD', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL',
                'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
                'NY', 'NM', 'OH', 'OK', 'OR', 'PA', 'RI', 'TN', 'TX', 'UT', 'VT', 'VA', 'WV', 'WA', 'WI', 'WY', 'DC']

df_usa=df_usgs[df_usgs.country_estate.apply(lambda x: x in state or x in abbreviation)]
print("Dimensión del dataset: ",df_usa.shape)
df_usa.head()

Dimensión del dataset:  (1332298, 10)


,time,latitude,longitude,depth,mag,magType,updated,place,horizontalError,country_estate
1,2013-01-01 00:10:51.379000+00:00,40.107400,-119.676800,14.600,-0.90,ml,2018-06-29 22:31:38.899000+00:00,"18 km NNW of Sutcliffe, Nevada",0.00,Nevada
2,2013-01-01 00:14:16.890000+00:00,51.975000,-169.566000,87.700,2.90,ml,2019-05-13 22:42:39.803000+00:00,"117 km SSW of Nikolski, Alaska",0.00,Alaska
3,2013-01-01 00:21:07.729000+00:00,61.683900,-147.402100,29.700,1.60,ml,2019-05-13 22:42:26.097000+00:00,"19 km SE of Glacier View, Alaska",0.00,Alaska
4,2013-01-01 00:24:33.234000+00:00,61.316700,-151.351100,73.300,1.30,ml,2019-05-13 22:42:40.207000+00:00,"24 km NW of Beluga, Alaska",0.00,Alaska
5,2013-01-01 00:29:01.520000+00:00,38.779335,-122.726997,1.905,0.42,md,2017-01-30 17:29:45.264000+00:00,"4 km S of Cobb, California",0.77,California


In [54]:
### Japón
df_japan=df_usgs[df_usgs.country_estate.str.contains("Japan")]
print("Dimensión del dataset: ",df_japan.shape)
df_japan.head()

Dimensión del dataset:  (11398, 10)


,time,latitude,longitude,depth,mag,magType,updated,place,horizontalError,country_estate
47,2013-01-01 03:32:13.010000+00:00,40.353,141.547,106.6,4.3,mb,2014-11-07 01:49:43.034000+00:00,"16 km SSE of Hachinohe, Japan",0.0,Japan
107,2013-01-01 09:19:16.080000+00:00,26.997,127.380,92.8,4.7,mb,2015-03-24 02:18:11.090000+00:00,"73 km NW of Nago, Japan",0.0,Japan
351,2013-01-02 10:31:02.520000+00:00,29.344,142.216,35.0,4.8,mb,2014-11-07 01:49:44.587000+00:00,"Izu Islands, Japan region",0.0,Japan region
369,2013-01-02 12:00:12.310000+00:00,42.159,143.314,68.5,4.2,mb,2014-11-07 01:49:44.604000+00:00,"80 km ESE of Shizunai-furukawach?, Japan",0.0,Japan
445,2013-01-02 20:31:04.080000+00:00,33.461,141.905,5.3,4.5,mb,2014-11-07 01:49:44.878000+00:00,"237 km SE of Katsuura, Japan",0.0,Japan


In [55]:
### Perú
df_peru=df_usgs[df_usgs.country_estate.str.contains("Peru")]
print("Dimensión del dataset: ",df_peru.shape)
df_peru.head()

Dimensión del dataset:  (2579, 10)


,time,latitude,longitude,depth,mag,magType,updated,place,horizontalError,country_estate
1581,2013-01-06 14:34:56.870000+00:00,-17.622,-72.014,74.7,4.7,mb,2014-11-07 01:49:47.177000+00:00,"55 km SSW of Punta de Bombón, Peru",0.0,Peru
2588,2013-01-09 18:38:45.120000+00:00,-18.011,-69.451,123.9,5.0,mb,2014-11-07 01:49:48.293000+00:00,"59 km ESE of Palca, Peru",0.0,Peru
4417,2013-01-15 22:34:50+00:00,-5.740,-80.660,40.0,4.6,mb,2022-05-03 16:03:40.239000+00:00,"27 km SE of Sechura, Peru",0.0,Peru
4442,2013-01-16 00:07:42.780000+00:00,-15.098,-70.802,57.1,4.6,mb,2014-11-07 01:49:50.648000+00:00,"11 km E of Ocuviri, Peru",0.0,Peru
4461,2013-01-16 01:29:22.110000+00:00,-15.864,-71.815,35.0,4.6,mb,2014-11-07 01:49:50.685000+00:00,"25 km SSW of Maca, Peru",0.0,Peru


### Dataset de Perú (IGP) - Período 01/01/2013 - now

In [ ]:
# Extracción de datos de la API del IGP
end_time=pd.to_datetime(datetime.now())
df_peru2=pd.read_excel(f"https://ultimosismo.igp.gob.pe/datos-sismicos-xls/01-01-2013/{end_time.strftime('%d-%m-%Y')}/-25.701/-1.396/-87.382/-65.624/2/1/9/0/900")

In [164]:
df_peru2

,fecha UTC,hora UTC,latitud (º),longitud (º),profundidad (km),magnitud (M)
0,01/01/2013,14:25:19.00,-12.6182,-76.7686,62,4.5
1,02/01/2013,06:09:04.00,-7.6207,-74.2491,172,4.6
2,02/01/2013,12:15:36.00,-5.3829,-78.6587,22,4.5
3,02/01/2013,15:52:36.00,-3.6134,-82.1250,32,4.5
4,03/01/2013,14:31:35.00,-15.0332,-74.0146,116,4.5
...,...,...,...,...,...,...
8021,27/12/2022,14:38:09,-11.5400,-76.9200,89,4.1
8022,23/2/2023,06:18:04,-8.6900,-77.8100,15,4.3
8023,3/3/2023,19:39:13,-6.8900,-80.2300,43,4.3
8024,27/3/2023,20:13:13,-10.5300,-74.7000,134,4.6


In [165]:
# Información del Dataset, incluyendo la cantidad de nulos por columna y el número total de registros y columnas.
df_peru2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8026 entries, 0 to 8025
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fecha UTC         8026 non-null   object 
 1   hora UTC          8026 non-null   object 
 2   latitud (º)       8026 non-null   float64
 3   longitud (º)      8026 non-null   float64
 4   profundidad (km)  8026 non-null   int64  
 5   magnitud (M)      8026 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 376.3+ KB


#### Diccionario de datos:
- 'fecha UTC': Esta columna indica la fecha del evento sísmico en formato UTC (Tiempo Universal Coordinado).

- 'hora UTC': Esta columna representa la hora exacta del evento sísmico en formato UTC. 

- 'latitud (º)': Esta columna indica la latitud geográfica del epicentro del sismo en grados decimales. La latitud es una coordenada geográfica que se refiere a la posición de un lugar en relación con el ecuador, medida en grados hacia el norte (+) o hacia el sur (-).

- 'longitud (º)': Esta columna representa la longitud geográfica del epicentro del sismo en grados decimales. La longitud es una coordenada geográfica que se refiere a la posición de un lugar en relación con el meridiano de referencia (generalmente el meridiano de Greenwich), medida en grados hacia el este (+) o hacia el oeste (-).

- 'profundidad (km)': Esta columna indica la profundidad a la que se produjo el sismo, medida en kilómetros desde la superficie terrestre. La profundidad proporciona información sobre la ubicación vertical del evento sísmico y puede ser un factor importante en la evaluación de su impacto.

- 'magnitud (M)': Esta columna representa la magnitud del sismo, que es una medida cuantitativa de la energía liberada durante el evento.

#### Creación de la columna `time` que contenga la fecha y hora en formato UTC

In [166]:
df_peru2["fecha UTC"]=pd.to_datetime(df_peru2["fecha UTC"],dayfirst=True)
df_peru2["hora UTC"]=pd.to_datetime(df_peru2["hora UTC"].apply(lambda x: x if "." in x  else x.strip()+".00"),format="%H:%M:%S.%f")
df_peru2.insert(0,"time",df_peru2["fecha UTC"].dt.strftime("%Y-%m-%dT") + df_peru2["hora UTC"].dt.strftime("%H:%M:%S.%fZ"))

# Eliminación de las columnas 'fecha UTC' y 'hora UTC'
df_peru2.drop(columns=["fecha UTC","hora UTC"],inplace=True)

# Conversión de la columna 'time' al tipo datetime
df_peru2["time"]=pd.to_datetime(df_peru2.time)

#### Creación de las columnas faltantes para el cruce de datos con el dataset `df_peru`

In [167]:
# Renombración de las columnas
df_peru2.rename(columns={"latitud (º)":"latitude","longitud (º)":"longitude",
                        "profundidad (km)":"depth","magnitud (M)":"mag"},inplace=True)

# Adición de las columnas faltantes
df_peru2["magType"]="mw"
df_peru2["updated"]=df_peru2.time
df_peru2["place"]="territory of Peru"
df_peru2["horizontalError"]=0.0
df_peru2["country_state"]="Peru"

In [168]:
# Visualización del resultado
df_peru2.head(3)

,time,latitude,longitude,depth,mag,magType,updated,place,horizontalError,country_state
0,2013-01-01 14:25:19+00:00,-12.6182,-76.7686,62,4.5,mw,2013-01-01 14:25:19+00:00,territory of Peru,0.0,Peru
1,2013-01-02 06:09:04+00:00,-7.6207,-74.2491,172,4.6,mw,2013-01-02 06:09:04+00:00,territory of Peru,0.0,Peru
2,2013-01-02 12:15:36+00:00,-5.3829,-78.6587,22,4.5,mw,2013-01-02 12:15:36+00:00,territory of Peru,0.0,Peru


#### Cruce de datos entre los datasets `df_peru` y `df_peru2`

In [169]:
""" 
OBSERVACIÓN: El campo en común será time, y se extraerán los datos de las variables 'updated', 'place' y 'horizontalError' 
para el dataset 'df_peru2', por ser el que tiene más registros.
"""

# Creación de una columna de fecha y hora truncada en minutos para realizar el merge
df_peru["time_min"]=df_peru["time"].dt.floor("min")
df_peru2["time_min"]=df_peru2["time"].dt.floor("min")

# Realización del merge basado en la columna 'time_min'
df_peru3=pd.merge(df_peru2,df_peru[['time_min','updated','place','horizontalError']],
                       on='time_min',how='left')

# Reemplazando
df_peru3["updated_x"]=df_peru3.apply(lambda x: x.updated_x if pd.isnull(x.updated_y) else x.updated_y,axis=1)
df_peru3["place_x"]=df_peru3.apply(lambda x: x.place_x if pd.isnull(x.place_y) else x.place_y,axis=1)
df_peru3["horizontalError_x"]=df_peru3.apply(lambda x: x.horizontalError_x if pd.isnull(x.horizontalError_y) else x.horizontalError_y,axis=1)

# Eliminación de columnas redundantes
df_peru3=df_peru3.drop(columns=["time_min","updated_y","place_y","horizontalError_y"])

# Renombrando las columnas necesarias
df_peru3.rename(columns={"updated_x":"updated","place_x":"place","horizontalError_x":"horizontalError"},inplace=True)

# Eliminación de los duplicados
df_peru3.drop_duplicates(inplace=True)
df_peru3=df_peru3.reset_index(drop=True)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16208\3683065914.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_peru["time_min"]=df_peru["time"].dt.floor("min")


In [170]:
# Visualización del resultado del cruce de datos
df_peru3

,time,latitude,longitude,depth,mag,magType,updated,place,horizontalError,country_state
0,2013-01-01 14:25:19+00:00,-12.6182,-76.7686,62,4.5,mw,2013-01-01 14:25:19+00:00,territory of Peru,0.00,Peru
1,2013-01-02 06:09:04+00:00,-7.6207,-74.2491,172,4.6,mw,2013-01-02 06:09:04+00:00,territory of Peru,0.00,Peru
2,2013-01-02 12:15:36+00:00,-5.3829,-78.6587,22,4.5,mw,2013-01-02 12:15:36+00:00,territory of Peru,0.00,Peru
3,2013-01-02 15:52:36+00:00,-3.6134,-82.1250,32,4.5,mw,2013-01-02 15:52:36+00:00,territory of Peru,0.00,Peru
4,2013-01-03 14:31:35+00:00,-15.0332,-74.0146,116,4.5,mw,2013-01-03 14:31:35+00:00,territory of Peru,0.00,Peru
...,...,...,...,...,...,...,...,...,...,...
8022,2023-06-24 14:23:53+00:00,-9.5600,-78.9500,45,4.0,mw,2023-06-24 14:23:53+00:00,territory of Peru,0.00,Peru
8023,2023-06-25 01:35:56+00:00,-15.6000,-74.1500,92,4.9,mw,2023-06-25 02:02:21.040000+00:00,Near the coast of southern Peru,9.64,Peru
8024,2023-06-25 13:26:50+00:00,-18.2300,-71.7400,28,4.4,mw,2023-06-25 13:43:20.040000+00:00,"63 km SSW of Pacocha, Peru",8.03,Peru
8025,2023-06-26 15:08:23+00:00,-9.8900,-75.5800,22,4.0,mw,2023-06-26 15:08:23+00:00,territory of Peru,0.00,Peru


### Creación de los datasets para las tablas ER

In [ ]:
#Dataset de Estados Unidos

df_usa.drop(columns="country_estate",inplace=True)
df_usa["id_country"]=1

In [ ]:
#Dataset de Japón

df_japan.drop(columns="country_estate",inplace=True)
df_japan["id_country"]=2

In [188]:
#Dataset de Perú

df_peru3.drop(columns="country_state",inplace=True)
df_peru3["id_country"]=3

In [194]:
# Dataset de Países

df_country=pd.DataFrame({"id_country":[1,2,3],
                         "country":["USA","Japan","Peru"]})

### Estandarización del tipo de magnitud de sismo

OBSERVACIONES:
* Algunas fórmulas de conversión de los tipos de magnitudes: [IGP](https://ultimosismo.igp.gob.pe/datos-sismicos), [USGS](https://www.usgs.gov/programs/earthquake-hazards/magnitude-types)
* Solo se aplicará la fórmula para el tipo de magnitud 'ml' y 'mb' debido a la falta de valores de parámetros para los demás tipos, a pesar de que algunos tienen fórmulas asociadas.
* De acuerdo con la USGS, los tipos de magnitudes 'mww', 'mwc', 'mwb' y 'mwr' se consideran equivalentes a 'mw', pero son medidas más precisas de la energía liberada en un sismo en comparación con otras escalas de magnitud.
* Después de estandarizar el tipo de magnitud de sismo, se considerarán solo los registros con el tipo de magnitud de sismo magnitud momento ('mw') debido a que es el estándar internacionalmente reconocido y utilizado para medir la energía liberada durante un terremoto.
* No se estandariza el dataset de Perú ya que, al cruzar los datos con el ``IGP``, se encuentra que todos los tipos de magnitud son reportados como ``mw``.

In [185]:
# Creación de una función para la estandarización
def apply_filter(df):
    filter = (df.magType == "ml") & (df.mag >= 3.3) & (df.mag <= 6.6)
    df["magType"] = df.magType.mask(filter, "mw")
    df["mag"] = df.mag.mask(filter, 0.8095 * df.mag + 1.3003)

    df["magType"] = df.magType.mask(df.magType.isin(["mww", "mwc", "mwb", "mwr"]), "mw")

    def filter2(mb):
        ms = 1.664 * mb - 3.753 if mb <= 5.9 else 2.763 * mb - 10.301
        if 3 < ms < 6.1:
            return 0.67 * ms + 2.07, True
        elif 6.2 < ms < 8.2:
            return 0.99 * ms + 0.08, True
        else:
            return mb, False
    mask = df["mag"].apply(filter2).apply(lambda x: x[1])
    df.loc[mask, "magType"] = "mw"
    df.loc[mask, "mag"] = df.loc[mask, "mag"].apply(filter2).apply(lambda x: x[0])
    
    # Redondenado 'depth', 'mag' y 'horizontalError' para que sean homogéneos con el dataset de Perú
    df["depth"]=df.depth.round().astype(int)
    df["mag"]=df.mag.round(1)
    df["horizontalError"]=df.horizontalError.round(2)

    return df[df.magType=="mw"]

In [186]:
# Estandarización de Estados Unidos y Japón

df_usa=apply_filter(df_usa)
df_japan=apply_filter(df_japan)

#### Exportación de los datasets en archivos CSV

In [ ]:
df_usa.to_csv("dataset_usa_etl.csv",index=False)
df_japan.to_csv("dataset_japan_etl.csv",index=False)
df_peru3.to_csv("dataset_peru_etl.csv",index=False)
df_country.to_csv("dataset_country_etl.csv",index=False)

####  **[Link de los Datasets Tratados](https://drive.google.com/drive/folders/1ujrPcokGmVWx-feOcKnkvmp1UN86B12Q?usp=drive_link)**

---